In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import xgboost

from functions import make_dataset

from functions import make_portfolio

from sklearn.utils import shuffle

import time

import pickle

In [2]:
features2 = pd.read_pickle("training_sets/features2")

In [3]:
Date2 = np.load("training_sets/Date2.npy", allow_pickle = True)

In [4]:
Tickers2 = np.load("training_sets/Tickers2.npy", allow_pickle = True)

In [5]:
with open('Training_window_test/top_50.pkl', 'rb') as f:
    top_50 = pickle.load(f)

In [6]:
cols = np.random.choice(top_50, 25, replace = False)

In [7]:
params1 = {
  'colsample_bynode': 0.6,
    'colsample_bytree' : 0.6,
    'colsample_bylevel' : 0.5,
  'learning_rate': 0.3,
  'max_depth': 3,
  'num_parallel_tree': 100,
  'objective': 'multi:softprob',
  'subsample': 0.9,
'num_class' : 2,
  'tree_method': 'gpu_hist',
    "verbosity" : 0,
    "lambda" : 1,
    "alpha" : 0
}

In [8]:
def make_dataset_eval(df, training_window, prediction_window, eval_size, prediction_date_number, Date_array, cols, pred_col):
    
    training_period = Date_array[prediction_date_number - training_window - prediction_window + 1: prediction_date_number - prediction_window +1]

    prediction_period = Date_array[prediction_date_number :prediction_date_number + eval_size + 1]
    
    X_train = shuffle(df.loc[training_period, np.append(cols, pred_col)])
    X_train = X_train.replace([np.inf, -np.inf], np.NaN)
    X_train = X_train.dropna(axis = 0)

    Y_train = X_train.pop(pred_col)
    
    X_test = shuffle(df.loc[prediction_period, np.append(cols, pred_col)])
    X_test = X_test.replace([np.inf, -np.inf], np.NaN)
    X_test = X_test.dropna(axis = 0)
    
    Y_test = X_test.pop(pred_col)
    
    return X_train, X_test, Y_train, Y_test

In [9]:
X_train, X_test, Y_train, Y_test = make_dataset_eval(features2, 200, 40, 35, 430, Date2, cols, "Class_binary_40_20")

In [10]:
weights = Y_train.map({0:19, 1:1})

In [13]:
xgtrain = xgboost.DMatrix(X_train.values, Y_train.values, feature_names = cols, weight = weights)

xgeval = xgboost.DMatrix(X_test.values, Y_test.values, feature_names = cols)

In [32]:
cvbst = xgboost.cv(params = params1, dtrain = xgtrain, num_boost_round = 10, nfold = 5, metrics = "mlogloss", verbose_eval=True, shuffle = False)

[0]	train-mlogloss:0.64311+0.00014	test-mlogloss:0.64349+0.00063
[1]	train-mlogloss:0.60923+0.00038	test-mlogloss:0.60999+0.00104
[2]	train-mlogloss:0.58512+0.00055	test-mlogloss:0.58620+0.00138
[3]	train-mlogloss:0.56703+0.00062	test-mlogloss:0.56841+0.00175
[4]	train-mlogloss:0.55278+0.00072	test-mlogloss:0.55444+0.00202
[5]	train-mlogloss:0.54165+0.00076	test-mlogloss:0.54356+0.00228
[6]	train-mlogloss:0.53262+0.00075	test-mlogloss:0.53476+0.00247
[7]	train-mlogloss:0.52513+0.00076	test-mlogloss:0.52748+0.00262
[8]	train-mlogloss:0.51876+0.00081	test-mlogloss:0.52135+0.00272
[9]	train-mlogloss:0.51328+0.00082	test-mlogloss:0.51609+0.00283


In [23]:
xgtrain.get_label()

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

In [30]:
xgtrain.get_uint_info('group_ptr')

array([], dtype=uint32)

In [31]:
xgtrain.num_row()

254456

In [25]:
predt = cvbst.predict(xgeval)

AttributeError: 'DataFrame' object has no attribute 'predict'

In [18]:
def return_metric(predt: np.ndarray, dtrain: xgboost.DMatrix):
    
    try:
        
        labels = dtrain.get_label()
        np.argsort(predt[:,0])[::-1][:100]

        c = np.mean[np.argsort(predt[:,0])[::-1][:100]])

        return 'avg_return', float(c)
    
    except:
        
        return "avg_return", float(np.nan)

In [ ]:
class my_callback(xgboost.callback.TrainingCallback):
    
    def __init__(self, name):
        self.name = name
        
    def after_iteration(self, model, epoch, evals_log):
        
        print()
        
            
        return False